In [7]:
#connect sql
import psycopg2
import pandas as pd
import numpy as np

# Load the existing CSV into DataFrames
df1 = pd.read_csv("Rajasthan.csv")
df2 = pd.read_csv("Telungana.csv")
df3 = pd.read_csv("Kadamba.csv")
df4 = pd.read_csv("West_Bengal.csv")  
df5 = pd.read_csv("Uttar_Pradesh.csv")
df6 = pd.read_csv("Chandigarh.csv")
df7 = pd.read_csv("Bihar.csv")
df8 = pd.read_csv("Andra.csv")
df9 = pd.read_csv("Assam.csv")
df10 = pd.read_csv("Kerala.csv")

# Concatenate all DataFrames
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)

# Clean up the Price column
df["Price"] = df["Price"].str.replace("INR", "")
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df["Price"].fillna(0, inplace=False)  # Removed 'inplace=True'

# Clean up the Star Rating column (if necessary)
df["Star_Rating"] = pd.to_numeric(df["Star_Rating"], errors="coerce")
df["Star_Rating"].fillna(0, inplace=False)  # Removed 'inplace=True'

# Replacing NaN values with None (for PostgreSQL compatibility)
df = df.replace({np.nan: None})

# Save the cleaned DataFrame to CSV
df.to_csv("Busdetails.csv", index=False)

# Database setup for PostgreSQL
def conn_dbsetup():
    conn = psycopg2.connect(
        host="localhost",
        database="projectdb",
        user="postgres",
        password="qwertyuiop"
    )
    cursor = conn.cursor()

    # Create table for bus details if it doesn't already exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bus_routes1 (
            id SERIAL PRIMARY KEY,
            route_name TEXT,
            route_link TEXT,
            bus_name TEXT,
            bus_type TEXT,
            departing_time VARCHAR(100),
            duration TEXT,
            reaching_time VARCHAR(100),
            star_rating FLOAT,
            price FLOAT,
            seat_availability INT
        )
    ''')

    conn.commit()
    return conn, cursor

# Insert DataFrame data into the PostgreSQL database
def insertingtodb(conn, cursor, df):
    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO bus_routes1
            (route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seat_availability)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            row['Route_Name'],
            row['Route_Link'],
            row['Bus_Name'],
            row['Bus_Type'],
            row['Departure'],
            row['Duration'],
            row['Arrival'],
            row['Star_Rating'],  # Already a float, no need to cast again
            row['Price'],  # Already a float, no need to cast again
            int(row['Seat_Availability']) if str(row['Seat_Availability']).isdigit() else 0
        ))

    conn.commit()

# Example of calling the functions
def sqlcsv():
    # Set up the PostgreSQL database
    conn, cursor =conn_dbsetup()

    # Insert the data from DataFrame into the PostgreSQL database
    insertingtodb(conn, cursor, df)

    # Close the connection
    cursor.close()
    conn.close()

# Run the function to load data into PostgreSQL
sqlcsv()
